# Sale Prediction

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [2]:
# load data
categories = pd.read_csv("item_categories.csv")
items = pd.read_csv("items.csv")
shops = pd.read_csv("shops.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("sales_train.csv")

From the competition, it wants the final result to be item id and the item count per month. The prediction primarily depends on item_price item_id and item_cnt_day. Firstly, we need to clean the data.

In [3]:
train = train.drop(columns=['date', 'item_price'])


In [4]:
train = pd.merge(train,test, on=['shop_id', 'item_id'])

In [5]:
train = train.drop(columns=['shop_id','item_id'])

In [6]:
train = train.sort_values(['ID', 'date_block_num'])

In [7]:
train.head()

,date_block_num,item_cnt_day,ID
896436,20,1.0,0
896437,22,1.0,0
896438,23,1.0,0
896439,23,1.0,0
896440,24,1.0,0


In [8]:
train.max()

date_block_num        33.0
item_cnt_day        2169.0
ID                214199.0
dtype: float64

In [25]:
train_by_month = train.groupby(['ID', 'date_block_num'],as_index=False).agg({'item_cnt_day':'sum'})

In [26]:
train_by_month.head()

,ID,date_block_num,item_cnt_day
0,0,20,1.0
1,0,22,1.0
2,0,23,2.0
3,0,24,2.0
4,0,28,1.0


In [127]:
train_by_month_short = train_by_month.loc[train_by_month['ID']<=5000]
train_by_month_short.head()

,ID,date_block_num,item_cnt_day
0,0,20,1.0
1,0,22,1.0
2,0,23,2.0
3,0,24,2.0
4,0,28,1.0


In [43]:
print(len(train_by_month_short))

11876


In [103]:
uniq_ID = train_by_month_short.ID.unique()
uniq_ID = np.array(["%d" % w for w in uniq_ID.reshape(uniq_ID.size)])
dic = {elem : pd.DataFrame for elem in uniq_ID}
for key in dic.keys():
    dic[key] = train_by_month_short[:][train_by_month_short.ID == int(key)]

In [117]:
for i in dic.keys():
    for j in range(34):
        if j not in dic[i].date_block_num.unique():
            dic[i] = dic[i].append({'ID': i, 'date_block_num': j}, ignore_index=True)
        

In [45]:
zero.head()

,ID,date_block_num,item_cnt_day
0,0.0,0.0,NaN
1,1.0,0.0,NaN
2,2.0,0.0,NaN
3,3.0,0.0,NaN
4,4.0,0.0,NaN


Check if the dataset is clean:

In [22]:
train.isna().sum()

date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [23]:
train.dtypes

date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object

Then group the data to get what we need to predict:

In [24]:
train_by_month = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_price':'mean','item_cnt_day':'sum'})
train_by_month.head(10)

item_price  item_cnt_day
date_block_num shop_id item_id                          
0              0       32            221.0           6.0
                       33            347.0           3.0
                       35            247.0           1.0
                       43            221.0           1.0
                       51            128.5           2.0
                       61            195.0           1.0
                       75             76.0           1.0
                       88             76.0           1.0
                       95            193.0           1.0
                       96             70.0           1.0